In [1]:
import pandas as pd


In [2]:
from groups.discrete_so2 import DiscreteSO2

input_config = {
    "C": 1,
    "H": 28,
    "W": 28
}

output_config = {
    "C": 10
}

experiment_config = {
    "num_iters": 1,
    "batch_size": 32,
    "max_epochs": 20,
    "log_every_n_steps": 100,
    "patience": 5,
    "rotate_train": True,
    "rotate_test": True,
    "seed": 420,
}


def get_gcnn_hparams(order: int, gconv_layers: int, channels: int, kernel_size: int = 5):
  return {
      "in_channels": input_config["C"],
      "out_channels": output_config["C"],
      "kernel_size": kernel_size,
      "gconv_layers": gconv_layers,
      "channels": channels,
      "group": DiscreteSO2(order=order)
  }


optimizer_hparams = {
    "lr": 1e-2,
    "weight_decay": 1e-4
}


In [3]:
metrics_base_path = "results/rotations/gcnns"


In [4]:
from modules.MNISTModule import MNISTModule
model_name = "GCNN"


In [5]:
from model_saving_helpers import get_model_save_name
from pathlib import Path
import numpy as np

from count_model_parameters import count_model_parameters


def aggregate_model_stats(model_name, model_hparams, optimizer_hparams, experiment_config):
  model_save_name = get_model_save_name(
      model_name=model_name,
      model_hparams=model_hparams,
      optimizer_hparams=optimizer_hparams,
      experiment_config=experiment_config
  )
  model = MNISTModule(model_name, model_hparams, optimizer_hparams).model
  model_params = count_model_parameters(model)
  million = pow(10, 3)

  model_metrics_per_run = Path(
      metrics_base_path).glob(f"*{model_save_name}*.csv")

  model_metrics = pd.concat([pd.read_csv(experiment_metrics)
                             for experiment_metrics in model_metrics_per_run])
  test_metrics_subselection = model_metrics[[
      "test_loss",
      "test_acc",
  ]].dropna()
  test_metrics_means = test_metrics_subselection.mean().round(4)
  test_metrics_stds = test_metrics_subselection.std().round(4)

  test_acc_mean = test_metrics_means["test_acc"]
  test_acc_std = test_metrics_stds["test_acc"]

  report = pd.DataFrame({
      "Discretization Order": [model_hparams["group"].order.item()],
      "Group Conv Layers": [model_hparams["gconv_layers"]],
      "Channels": [model_hparams["channels"]],
      "Kernel Size": [model_hparams["kernel_size"]],
      "Params": [f"{np.round((model_params / million), 2)} K"],
      "Test Accuracy": [f"{test_acc_mean} ± {test_acc_std}"],
  })

  return report


In [6]:
from model_saving_helpers import get_model_save_name
from pathlib import Path

orders = [4, 8]
convs = [2, 4]
channels = [16, 32]

all_models_stats = []

for order in orders:
  for conv in convs:
    for channel in channels:
      model_save_name = get_model_save_name(
          model_name=model_name,
          model_hparams=get_gcnn_hparams(order, conv, channel),
          optimizer_hparams=optimizer_hparams,
          experiment_config=experiment_config
      )
      model_metrics_per_run = Path(
          metrics_base_path).glob(f"*{model_save_name}*.csv")
      if len(list(model_metrics_per_run)):
        all_models_stats.append(
            aggregate_model_stats(
                model_name=model_name,
                model_hparams=get_gcnn_hparams(order, conv, channel),
                optimizer_hparams=optimizer_hparams,
                experiment_config=experiment_config
            )
        )

all_models_stats = pd.concat(all_models_stats)

all_models_stats


,Discretization Order,Group Conv Layers,Channels,Kernel Size,Params,Test Accuracy
0,4,2,16,5,51.77 K,0.9356 ± 0.0066
0,4,2,32,5,205.93 K,0.9507 ± 0.0038
0,4,4,16,5,102.97 K,0.9802 ± 0.0019
0,4,4,32,5,410.73 K,0.9801 ± 0.0078
0,8,2,16,5,102.97 K,0.9377 ± 0.0048
0,8,2,32,5,410.73 K,0.9405 ± 0.0027
0,8,4,16,5,205.37 K,0.9827 ± 0.001
0,8,4,32,5,820.33 K,0.9855 ± 0.0007


In [7]:
print(all_models_stats.to_latex(
    column_format="c|c|c|c|c|c",
    index=False)
)


\begin{tabular}{c|c|c|c|c|c}
\toprule
 Discretization Order &  Group Conv Layers &  Channels &  Kernel Size &   Params &   Test Accuracy \\
\midrule
                    4 &                  2 &        16 &            5 &  51.77 K & 0.9356 ± 0.0066 \\
                    4 &                  2 &        32 &            5 & 205.93 K & 0.9507 ± 0.0038 \\
                    4 &                  4 &        16 &            5 & 102.97 K & 0.9802 ± 0.0019 \\
                    4 &                  4 &        32 &            5 & 410.73 K & 0.9801 ± 0.0078 \\
                    8 &                  2 &        16 &            5 & 102.97 K & 0.9377 ± 0.0048 \\
                    8 &                  2 &        32 &            5 & 410.73 K & 0.9405 ± 0.0027 \\
                    8 &                  4 &        16 &            5 & 205.37 K &  0.9827 ± 0.001 \\
                    8 &                  4 &        32 &            5 & 820.33 K & 0.9855 ± 0.0007 \\
\bottomrule
\end{tabular}



/var/folders/qw/52cxc2651nb6vmjvb6vb0gmr0000gn/T/ipykernel_32982/985651510.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(all_models_stats.to_latex(


In [8]:
import re


def get_max_model_stats(model_name, model_hparams, optimizer_hparams, experiment_config):
  model_save_name = get_model_save_name(
      model_name=model_name,
      model_hparams=model_hparams,
      optimizer_hparams=optimizer_hparams,
      experiment_config=experiment_config
  )
  model = MNISTModule(model_name, model_hparams, optimizer_hparams).model
  model_params = count_model_parameters(model)
  million = pow(10, 3)

  model_metrics_per_run = Path(
      metrics_base_path).glob(f"*{model_save_name}*.csv")

  all_metrics_files = []
  for metrics in model_metrics_per_run:
    run = re.search("-run=.", str(metrics)).group(0)[-1]
    read_metrics = pd.read_csv(metrics)
    read_metrics["run"] = int(run)
    all_metrics_files.append(read_metrics.reset_index())

  model_metrics = pd.concat(all_metrics_files)
  test_metrics_subselection = model_metrics[[
      "run",
      "test_loss",
      "test_acc",
  ]].dropna()

  best_acc_row = test_metrics_subselection[test_metrics_subselection.test_acc ==
                                           test_metrics_subselection.test_acc.max()].reset_index()

  report = pd.DataFrame({
      "Discretization Order": [model_hparams["group"].order.item()],
      "Group Conv Layers": [model_hparams["gconv_layers"]],
      "Channels": [model_hparams["channels"]],
      "Kernel Size": [model_hparams["kernel_size"]],
      "Params": [f"{np.round((model_params / million), 2)} K"],
      "Best Run": [best_acc_row["run"].tolist()[0]],
      "Max Test Accuracy": [best_acc_row["test_acc"].tolist()[0]],
  })

  return report


In [9]:
from model_saving_helpers import get_model_save_name
from pathlib import Path

orders = [4, 8]
convs = [2, 4]
channels = [16, 32]

all_models_stats = []

for order in orders:
  for conv in convs:
    for channel in channels:
      model_save_name = get_model_save_name(
          model_name=model_name,
          model_hparams=get_gcnn_hparams(order, conv, channel),
          optimizer_hparams=optimizer_hparams,
          experiment_config=experiment_config
      )
      model_metrics_per_run = Path(
          metrics_base_path).glob(f"*{model_save_name}*.csv")
      if len(list(model_metrics_per_run)):
        all_models_stats.append(
            get_max_model_stats(
                model_name=model_name,
                model_hparams=get_gcnn_hparams(order, conv, channel),
                optimizer_hparams=optimizer_hparams,
                experiment_config=experiment_config
            )
        )

all_models_stats = pd.concat(all_models_stats)

all_models_stats


,Discretization Order,Group Conv Layers,Channels,Kernel Size,Params,Best Run,Max Test Accuracy
0,4,2,16,5,51.77 K,2,0.9454
0,4,2,32,5,205.93 K,2,0.9530
0,4,4,16,5,102.97 K,3,0.9817
0,4,4,32,5,410.73 K,1,0.9861
0,8,2,16,5,102.97 K,1,0.9432
0,8,2,32,5,410.73 K,3,0.9433
0,8,4,16,5,205.37 K,3,0.9837
0,8,4,32,5,820.33 K,1,0.9861


In [10]:
print(all_models_stats.to_latex(
    column_format="c|c|c|c|c|c",
    index=False)
)


\begin{tabular}{c|c|c|c|c|c}
\toprule
 Discretization Order &  Group Conv Layers &  Channels &  Kernel Size &   Params &  Best Run &  Max Test Accuracy \\
\midrule
                    4 &                  2 &        16 &            5 &  51.77 K &         2 &             0.9454 \\
                    4 &                  2 &        32 &            5 & 205.93 K &         2 &             0.9530 \\
                    4 &                  4 &        16 &            5 & 102.97 K &         3 &             0.9817 \\
                    4 &                  4 &        32 &            5 & 410.73 K &         1 &             0.9861 \\
                    8 &                  2 &        16 &            5 & 102.97 K &         1 &             0.9432 \\
                    8 &                  2 &        32 &            5 & 410.73 K &         3 &             0.9433 \\
                    8 &                  4 &        16 &            5 & 205.37 K &         3 &             0.9837 \\
                 

/var/folders/qw/52cxc2651nb6vmjvb6vb0gmr0000gn/T/ipykernel_32982/985651510.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(all_models_stats.to_latex(
